# 🥇 주조 결함 검사: 4대 딥러닝 모델 성능 비교 (Kaggle 전용 통합본)

이 노트북은 **외부 파일(src 폴더 등) 없이도** 이 파일 하나만으로 4가지 모델(**ResNet18, DenseNet121, EfficientNet-B0, MobileNetV2**)을 학습하고 비교할 수 있도록 모든 코드를 통합했습니다.

### 📌 작업 환경 설정
1. **Accelerator**: GPU P100 (권장)
2. **Internet**: ON
3. **Dataset**: `Casting Product Image Data`가 추가되어 있어야 함

### 📁 결과물 위치
모든 결과물(그래프, 모델 파일)은 **`/kaggle/working`** 에 바로 저장되어 우측 Output 탭에서 즉시 확인 가능합니다.

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.metrics import roc_curve, auc

# --- [1. 전역 설정] ---
# 캐글 데이터셋 경로 (데이터셋 구조에 따라 조절될 수 있음)
DATA_DIR = "/kaggle/input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data"
OUTPUT_DIR = Path("/kaggle/working")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 5  # 원하시는 대로 조절 가능
LR = 0.001

print(f"✅ Device: {DEVICE}")
print(f"✅ Data Path: {DATA_DIR}")

In [ ]:
# --- [2. 데이터셋 클래스 정의] ---
class CastingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.classes = ['ok_front', 'def_front']
        self.image_paths = []
        self.labels = []
        
        for i, cls in enumerate(self.classes):
            cls_path = self.root_dir / cls
            # jpeg와 jpg 모두 지원
            exts = ['*.jpeg', '*.jpg', '*.png']
            for ext in exts:
                for img_path in cls_path.glob(ext):
                    self.image_paths.append(img_path)
                    self.labels.append(i)
                    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# 데이터 변환 (Augmentation 포함)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),    # 상하 반전 (New)
    transforms.RandomRotation(45),           # 회전 45도 (Stronger)
    transforms.ColorJitter(brightness=0.3, contrast=0.3), # 밝기/대비 변화 (New)
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), # 이동/스케일 (New)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 로더 생성
train_set = CastingDataset(os.path.join(DATA_DIR, "train"), transform=train_transform)
test_set = CastingDataset(os.path.join(DATA_DIR, "test"), transform=test_transform)

# num_workers를 0으로 설정하면 프로세스 충돌 에러가 발생하지 않습니다.
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"📦 데이터 로드 완료: Train({len(train_set)}), Test({len(test_set)})")

In [ ]:
# --- [3. 4가지 모델 아키텍처 정의] ---
def build_model(model_name):
    print(f"🛠️ Building {model_name}...")
    if model_name == 'resnet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif model_name == 'densenet121':
        model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        model.classifier = nn.Linear(model.classifier.in_features, 2)
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    elif model_name == 'mobilenet_v2':
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    return model.to(DEVICE)

print("✅ 모델 생성 함수 준비 완료")

In [ ]:
# --- [4. 학습 프로토콜 정의] ---
def run_training(model_name, model, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0.0
    
    for epoch in range(EPOCHS):
        # Training Phase
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        pbar = tqdm(train_loader, desc=f"[{model_name}] Epoch {epoch+1}/{EPOCHS}", leave=False)
        for imgs, labels in pbar:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)
            
        epoch_loss = running_loss / total
        epoch_acc = correct.double() / total
        
        # Test/Validation Phase
        model.eval()
        current_val_loss, val_correct, val_total = 0.0, 0, 0
        current_val_probs = []
        current_val_labels = []
        
        with torch.no_grad():
            for imgs, labels in test_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                current_val_loss += loss.item() * imgs.size(0)
                
                probs = torch.softmax(outputs, dim=1)[:, 1]
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)
                val_total += labels.size(0)
                
                current_val_probs.extend(probs.cpu().numpy())
                current_val_labels.extend(labels.cpu().numpy())

        v_loss = current_val_loss / val_total
        v_acc = val_correct.double() / val_total
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        history['val_loss'].append(v_loss)
        history['val_acc'].append(v_acc.item())
        
        print(f"Epoch {epoch+1}: Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | Val Acc: {v_acc:.4f} | Val Loss: {v_loss:.4f}")
        
        # 모델 저장
        if v_acc > best_acc:
            best_acc = v_acc
            torch.save(model.state_dict(), OUTPUT_DIR / f"{model_name}_best.pth")
            # ROC 계산을 위해 최고 성능일 때의 확률과 라벨 저장
            history['best_val_probs'] = current_val_probs
            history['best_val_labels'] = current_val_labels
            
    return history

print("✅ 학습 엔진 준비 완료")

In [ ]:
# --- [5. 실제 비교 학습 실행] ---
models_list = ['resnet18', 'densenet121', 'efficientnet_b0', 'mobilenet_v2']
results = {}

for name in models_list:
    m = build_model(name)
    results[name] = run_training(name, m, train_loader, test_loader)
    print(f"⭐ {name} 완료!")

print("\n🎊 모든 실험이 종료되었습니다.")

In [ ]:
# --- [6. 최종 결과 시각화] ---
fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# 1. Accuracy Comparison
for name, hist in results.items():
    axes[0].plot(hist['val_acc'], label=f"{name} (Best: {max(hist['val_acc']):.4f})")
axes[0].set_title("Model Comparison - Validation Accuracy", fontsize=14)
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy")
axes[0].legend()
axes[0].grid(True, linestyle='--')

# 2. Loss Comparison
for name, hist in results.items():
    axes[1].plot(hist['val_loss'], label=f"{name} (Min: {min(hist['val_loss']):.4f})")
axes[1].set_title("Model Comparison - Validation Loss", fontsize=14)
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Loss")
axes[1].legend()
axes[1].grid(True, linestyle='--')

# 3. ROC Curve Comparison (Best Model State)
for name, hist in results.items():
    if 'best_val_labels' in hist:
        fpr, tpr, _ = roc_curve(hist['best_val_labels'], hist['best_val_probs'])
        roc_auc = auc(fpr, tpr)
        axes[2].plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.4f})")

axes[2].plot([0, 1], [0, 1], 'k--', label='Random')
axes[2].set_title("ROC Curve Comparison (Best Model)", fontsize=14)
axes[2].set_xlabel("False Positive Rate")
axes[2].set_ylabel("True Positive Rate")
axes[2].legend(loc='lower right')
axes[2].grid(True, linestyle='--')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "model_performance_comparison.png")
plt.show()

# 요약 테이블 출력
summary = []
for name, hist in results.items():
    summary.append({'Model': name, 'Best Acc': max(hist['val_acc']), 'Final Acc': hist['val_acc'][-1]})

df_summary = pd.DataFrame(summary)
df_summary.to_csv(OUTPUT_DIR / "model_results.csv", index=False)
print(df_summary)

print(f"\n💾 모든 결과 파일(.pth, .png, .csv)이 {OUTPUT_DIR} 에 저장되었습니다.")

In [ ]:
# --- [7. 최종 우승 모델(EfficientNet-B0) 집중 학습] ---
# 4중 철벽 방어 (Dropout, Early Stopping, Checkpoint, Scheduler) 적용

# 1. 모델 재정의 (Dropout 추가)
final_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# 기존 Classifier의 구조를 유지하되 Dropout 추가
# EfficientNet의 classifier는 보통 [Dropout, Linear] 구조임. 더 강력하게 설정.
final_model.classifier = nn.Sequential(
    nn.Dropout(p=0.5), # 강력한 Dropout (50%)
    nn.Linear(1280, 2) # EfficientNet-B0의 마지막 채널은 1280
)
final_model = final_model.to(DEVICE)

# 2. 학습 설정 (Scheduler & Early Stopping 준비)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=0.0005) # 조금 더 신중한 학습률
# 성능이 정체되면 학습률을 1/10로 줄임
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Early Stopping 설정
patience_limit = 5
patience_check = 0
best_loss = float('inf')

print("🚀 최종 모델(EfficientNet-B0) 심화 학습 시작...")

FINAL_EPOCHS = 20 # 충분히 길게 잡지만 Early Stopping으로 조기 종료됨
final_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(FINAL_EPOCHS):
    # Train
    final_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = final_model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
        total += labels.size(0)
    
    # Validation
    final_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = final_model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            val_total += labels.size(0)
            
    epoch_train_loss = running_loss / len(train_set)
    epoch_val_loss = val_loss / val_total
    epoch_train_acc = correct.double() / total
    epoch_val_acc = val_correct.double() / val_total
    
    final_history['train_loss'].append(epoch_train_loss)
    final_history['train_acc'].append(epoch_train_acc.item())
    final_history['val_loss'].append(epoch_val_loss)
    final_history['val_acc'].append(epoch_val_acc.item())
    
    print(f"Epoch {epoch+1}: Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")
    
    # Scheduler Step
    scheduler.step(epoch_val_loss)
    
    # Checkpoint & Early Stopping
    if epoch_val_loss < best_loss:
        best_loss = epoch_val_loss
        patience_check = 0
        torch.save(final_model.state_dict(), OUTPUT_DIR / "final_efficientnet_b0.pth")
        print(f"  ✅ Best Model Saved (Loss: {best_loss:.4f})")
    else:
        patience_check += 1
        print(f"  ⚠️ 성능 개선 없음 ({patience_check}/{patience_limit})")
        if patience_check >= patience_limit:
            print("🛑 Early Stopping 발동! 학습을 종료합니다.")
            break

print("🏁 최종 학습 완료!")

In [ ]:
# --- [7-1. 최종 학습 결과 시각화] ---
# 학습 과정에서의 Loss와 Accuracy 변화 변화를 그래프로 확인합니다.

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 6))

# 1. Loss Curve
plt.subplot(1, 2, 1)
plt.plot(final_history['train_loss'], label='Train Loss', marker='.')
plt.plot(final_history['val_loss'], label='Val Loss', marker='.')
plt.title('Final Model Training - Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--')

# 2. Accuracy Curve
plt.subplot(1, 2, 2)
plt.plot(final_history['train_acc'], label='Train Acc', marker='.')
plt.plot(final_history['val_acc'], label='Val Acc', marker='.')
plt.title('Final Model Training - Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "final_training_history.png")
plt.show()

print(f"📈 학습 그래프가 저장되었습니다: {OUTPUT_DIR / 'final_training_history.png'}")

In [ ]:
# --- [8. 최종 결과 심층 분석: 오차 행렬 & 실패 사례] ---
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 1. Best Model 로드
final_model.load_state_dict(torch.load(OUTPUT_DIR / "final_efficientnet_b0.pth"))
final_model.eval()

all_preds = []
all_labels = []
failed_samples = [] # 틀린 이미지 저장용

print("🔍 전체 테스트 데이터 검증 중...")
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(DEVICE)
        outputs = final_model(imgs)
        _, preds = torch.max(outputs, 1)
        
        for i in range(len(preds)):
            pred = preds[i].item()
            label = labels[i].item()
            all_preds.append(pred)
            all_labels.append(label)
            
            if pred != label:
                # 틀린 경우: 이미지(tensor)를 cpu로 옮겨서 저장
                failed_samples.append((imgs[i].cpu(), label, pred))

# 2. Confusion Matrix 시각화
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Defect'], yticklabels=['Normal', 'Defect'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Final Model)')
plt.savefig(OUTPUT_DIR / "final_confusion_matrix.png")
plt.show()

print(f"📊 분류 결과 요약:")
print(f" - 전체 데이터: {len(all_labels)}개")
print(f" - 틀린 개수: {len(failed_samples)}개")
print(f" - 최종 정확도: {100 * (len(all_labels)-len(failed_samples))/len(all_labels):.2f}%")

# 3. 실패 사례(오답) 시각화 (최대 5개)
if len(failed_samples) > 0:
    print("\n❌ 오답 노트 (틀린 케이스 확인)")
    num_show = min(5, len(failed_samples))
    fig, axes = plt.subplots(1, num_show, figsize=(15, 3))
    if num_show == 1: axes = [axes]
    
    for i in range(num_show):
        img_tensor, true_label, pred_label = failed_samples[i]
        # Tensor -> Image 변환 (Unnormalize)
        img_np = img_tensor.permute(1, 2, 0).numpy()
        img_np = img_np * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
        img_np = np.clip(img_np, 0, 1)
        
        axes[i].imshow(img_np)
        axes[i].set_title(f"True: {true_label} | Pred: {pred_label}", color='red')
        axes[i].axis('off')
    plt.show()
else:
    print("🎉 와우! 틀린 문제가 하나도 없습니다. 완벽합니다!")